In [5]:
from functools import reduce
import wandb
import pandas as pd

figure_dir = "/home/damow/Dropbox/Apps/Overleaf/Unsupervised Optimal Power Flow Using Graph Neural Networks/figures"

In [6]:
api = wandb.Api()
runs = api.runs("damowerko/opf", filters={"tags": "sweep1"})
data = [{**run.config, **run.summary} for run in runs]
df = pd.DataFrame(data)
df = df.rename(columns={"gnn_layers": "L", "lr": "$\eta$"})

In [7]:
parameters = ["s", "t", "K", "F", "L", "$\eta$"]
best_idx = df[df["test/inequality/rate"] < 0.01]["test/cost"].idxmin()
best_parameters = {name: df.iloc[best_idx][name] for name in parameters}
display(best_parameters)

{'s': 10, 't': 500, 'K': 8, 'F': 32, 'L': 2, '$\\eta$': 0.0001}

In [9]:
value = ["test/cost", "test/inequality/rate"]


def pivot(param1, param2):
    selected = []
    for param_name, param_value in best_parameters.items():
        if param_name in (param1, param2):
            continue
        selected.append(df[param_name] == param_value)
    selected = reduce(lambda a, b: a & b, selected)
    return df[selected].pivot(param1, param2, value).rename(
        columns={"test/cost": "Generation Cost", "test/inequality/rate": "Constraint Violation Rate"}
    )


tables = [
    pivot("K", "F"),
    pivot("s", "t"),
    pivot("L", "$\eta$").rename(columns={0.00001: "1e-5", 0.0001: "1e-4", 0.001: "1e-3"})
]

for table in tables:
    display(table)
    file = f"{figure_dir}/search_{table.columns.names[-1]}{table.index.name}.tex"
    file = file.replace("$\eta$", "eta")
    table.to_latex(
        buf=file,
        float_format="%0.2f",
        escape=False,
        multicolumn_format="c"
    )


Generation Cost                     Constraint Violation Rate            \
F              16        32        64                        16        32   
K                                                                           
2        4.593867  1.321298  1.162804                  0.090623  0.044089   
4        3.834059  1.153386  1.213398                  0.087791  0.072201   
8        1.382331  1.190462  1.156499                  0.006015  0.009333   

             
F        64  
K            
2  0.076460  
4  0.087084  
8  0.050981

Generation Cost                     Constraint Violation Rate            \
t               10        100       500                       10        100   
s                                                                             
10         1.151249  1.167303  1.190462                  0.039353  0.068545   
100        1.363101  1.347402  1.438628                  0.026241  0.002834   
500        1.326278  1.280657  1.571965                  0.033379  0.038392   

               
t         500  
s              
10   0.009333  
100  0.018948  
500  0.068630

Generation Cost                     Constraint Violation Rate  \
$\eta$            1e-5      1e-4      1e-3                      1e-5   
L                                                                      
1             4.270278  3.990360  4.048145                  0.087954   
2             4.880285  1.190462  1.177610                  0.093259   

                            
$\eta$      1e-4      1e-3  
L                           
1       0.087791  0.087791  
2       0.009333  0.046243